<a href="https://colab.research.google.com/github/sbickler/Diff_Gene_Expression/blob/master/GenePathwayCovering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input and Output Data Structures

*   ***pathways***: a dictionary (P -> S) which describes a set of genes S which are involved in a pathway P
*   ***genes***: a dictionary (G -> S) which describes a set of pathways S which contain a gene G
*   ***covering***: the set which approximates the smallest collection of genes that can produce a span which contains all pathways 

Given a ***pathways*** or ***genes*** object as an input, the other respective data structure can be easily created with $O(P + E)$ or $O(G + E)$ time complexity, with $P$, $G$, and $E$ denoting the number of pathways, genes, and edges.

The ***pathways*** object serves a double purpose to store only the pathways which are not yet included in the span. Similarly, the ***genes*** object only stores genes which are not included in ***covering***.

NOTE: to avoid confusion all italicized, bolded words refer to the objects themselves


In [ ]:
pathways = {"A": {1, 2, 3, 4},
            "B": {1, 4},
            "C": {2, 3},
            "D": {1},
            "E": {3, 4},
            "F": {1}}

genes = {1: {"A", "B", "D", "F"},
         2: {"A", "C"},
         3: {"A", "C", "E"},
         4: {"A", "B", "E"}}

covering = set()

In [ ]:
'''Converts pathways to genes or vica versa.'''
def convert(input):
  output = {}
  for key in input:
    value_set = input[key]
    for value in value_set:
      if output.get(value) == None:
        output[value] = set()
      output[value].add(key)
  return output

# Pseudocode

1.   Search for pathways which only are associated with a single gene
2.   If such pathways are found, then 

    a) the genes associated with those pathways must be used in the span

    b) genes are added to the covering and removed from ***genes***

    c) any other pathways associated with the genes are already included in the span and can also be removed from search by removing them from ***pathways***

3.   Greedy algorithm begins

    a) a greedy search finds the gene with the greatest number of associated pathways not yet contained in the span i.e. remaining in ***genes***

    b) the gene is added to the covering and removed from ***genes***
    
    c) any other pathways associated with the gene are removed similar to step 2c)

    d) repeat step 3 until all pathways are removed

In [ ]:
# Step 1
temp_genes = set()
for p in pathways:
  g = pathways[p]
  if len(g) == 1:
    temp_genes.update(g)

# Step 2
if len(temp_genes) != 0:
  covering.update(temp_genes)
  for g in temp_genes:
    del genes[g]
  for p in list(pathways.keys()):
    if len(temp_genes.intersection(pathways[p])) > 0:
      del pathways[p]

# Step 3
while bool(pathways):
  remaining = set(pathways.keys())
  temp_gene = None
  max_count = 0
  for g in genes:
    p = genes[g]
    overlap = len(remaining.intersection(p))
    if overlap > max_count:
      max_count = overlap
      temp_gene = g
  if temp_gene == None:
    raise Exception("No solution.")
  else:
    covering.add(temp_gene)
    del genes[temp_gene]
    for p in list(pathways.keys()):
      if temp_gene in pathways[p]:
        del pathways[p]

print(covering)

{1, 3}


# Algorithm Details

A brute force algorithm which finds the exact, optimal collection of genes which can span all pathways has an exponential complexity. Thus, for large datasets, the only feasible way to solve this problem is through approximation via the greedy algorithm.

At each step, the greedy algorithm selects the gene which is associated with the most pathways not yet contained in the span. Of course, this may not always result in the most optimal solultion but is significantly less computationally taxing compared to a brute force approach. As the size of the dataset increases, the greedy approximation is more accurate relative to smaller datasets.

Steps 1 and 2 are designed to improve upon the approximation by identifying the cases in which specific genes MUST be included to span all pathways. This occurs when there is only a single linkage, or association, between a particular gene and pathway. While the exact probability of this occuring is difficult to compute, it is clear that this approximation improvement becomes less effective for larger datasets as the probability that at least one occurance of this event decreases. 